In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

In [62]:
#Import the dataset
data_train=pd.read_csv('/content/drive/MyDrive/N12-Kĩ thuật giấu tin/DataTest/LSTM2/Train_56000.csv')
data_test=pd.read_csv('/content/drive/MyDrive/N12-Kĩ thuật giấu tin/DataTest/LSTM2/Test_14000.csv')

X_train=data_train.iloc[:56000,[1,2,3,4,5]].values
Y_train=data_train.iloc[:56000,0].values

X_test=data_test.iloc[:14000,[1,2,3,4,5]].values
Y_test=data_test.iloc[:14000,0].values


In [63]:
X_train, Y_train, X_test, Y_test = list(map(lambda x: np.array(x, dtype=np.float64), [X_train, Y_train, X_test, Y_test]))

In [64]:
data_train.head()

,LABEL,ip.proto,ip.id,tcp.seq_raw,frame.len,frame.cap_len
0,1,1,39222,0,74,74
1,0,6,57919,4267710083,2962,66
2,1,6,30277,1082759981,266,266
3,1,6,33153,1083194381,166,166
4,0,6,62841,3618095064,79,66


In [65]:
X_train[0]

array([1.0000e+00, 3.9222e+04, 0.0000e+00, 7.4000e+01, 7.4000e+01])

In [66]:
max_words = 65536
max_len = 5
# tok = Tokenizer(num_words=max_words)
# tok.fit_on_texts(X_train)
# sequences = tok.texts_to_sequences(X_train)
#sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
sequences_matrix = X_train
Y_train = Y_train.reshape(-1, 1)

In [67]:
tcp_train = X_train[:, 2]
tcp_test = X_test[:, 2]

In [68]:
#Trộn train+test
tcp = np.concatenate([tcp_train, tcp_test])

In [69]:
tcp.shape

(70000,)

In [70]:
#Sort tcp_test
tcp_set = list(set(tcp))
tcp_sort = list(np.sort(tcp_set))
len(tcp_set)

47681

In [71]:
#Đánh index value tcp_train và tcp_test= index trong tcp+sort
result_train = []
result_test = []
for value in tcp_train:
  index = tcp_sort.index(value)
  result_train.append(index)

for value in tcp_test:
  index = tcp_sort.index(value)
  result_test.append(index)

result_train = np.array(result_train)
result_test = np.array(result_test)

In [ ]:
1, int -> smaller
2, 8, 9 -> 1, 2

In [72]:
np.max(result_test)

47680

In [73]:
X_train[:, 2] = result_train
X_test[:, 2] = result_test

In [74]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    # layer = Merge([max_len, context_model], mode="dot", dot_axes=0)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [42]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 5)]               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 5, 50)             3276800   
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_6 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=1024,epochs=10,
          validation_split=0.2)

In [56]:
model.evaluate(X_test, Y_test)

438/438 [==============================] - 1s 3ms/step - loss: 0.1004 - accuracy: 0.9714


[0.10040238499641418, 0.9713571667671204]

In [ ]:
model.predict(np.array([X_test[4]]))

array([[0.9999355]], dtype=float32)

In [ ]:
Y_test[4]

1.0